In [474]:
#these are libraries we need
from splinter import Browser
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from bs4 import BeautifulSoup as soup 
import re
import pandas as pd
import matplotlib.pyplot as plt
import time

#here we are setting up opening an incognito tab
chrome_options = Options()
chrome_options.add_argument("--incognito")

#here we are setting up the browser
browser = Browser("chrome", options = chrome_options)

#this is the base url
#this is just up to the search part without the filters
base_url = "https://www.facebook.com/marketplace/103097919731105/vehicles?"

#this is our search parameters basically your filters
#if we want more specific you can add
transmision = ""
make = "BMW"
model = "340"
days_listed = "1"
min_price = 200
max_price = 50000
min_mileage = 20000
max_mileage = 170000
min_year = 1980
max_year = 2024


#now here is the new url with our filters
url = (f"{base_url}minPrice={min_price}&maxPrice={max_price}&minMileage={min_mileage}&maxMileage={max_mileage}&minYear={min_year}&maxYear={max_year}&make={make}&model={model}")

#this is just visiting the website now
browser.visit(url)

#here we need to wait for the login button to be clickable we give it 15 seconds
wait = WebDriverWait(browser.driver, 15)
#once the button pops up
login_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="login_popup_cta_form"]/div/div[5]/div')))
#then we will click the button
login_button.click()

#now here is information
UserID = "put yours"
PassID = "put yours" 

#here is finding the id and filling in
email_field = browser.driver.find_element(By.XPATH, '//*[@id="email"]')
email_field.send_keys(UserID)

password_field = browser.driver.find_element(By.XPATH, '//*[@id="pass"]')
password_field.send_keys(PassID)

#then finally clicking log in again
login_button_final = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="loginbutton"]')))
login_button_final.click()

#this is how many times I want to scroll
scroll_count = 10

#this is how long I wait so it loads
scroll_delay = 5

for i in range(scroll_count):
    #the zero here represents vertical scrolling
    #document body tells it to scroll the ENTIRE length
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    #here is the scroll delay so it can load
    time.sleep(scroll_delay)

#now we are going to check through the HTMLS
html = browser.html

#here is an object for beautiful soup
market_soup = soup(html, "html.parser")

#now heres how we check if it scraped correctly
market_soup

#here now that we have the html information close the tab
#browser.quit()
#now we are going to be extracting all the information into lists
#organization step

#first getting the titles
#get the class name and put it in here
titles_div = market_soup.find_all("span", class_= "x1lliihq x6ikm8r x10wlt62 x1n2onr6")
#we are doing this because the css is involed so we will look for the title in the titles div and then strip it so its only the title
#I had to add extra beacuse I kept getting those in the beggining
titles_list = [title.text.strip() for title in titles_div if title.text.strip() not in ["Filters", "Categories", "Shop by Category"]]

#this is for the price
prices_div =  market_soup.find_all("span", class_= "x193iq5w xeuugli x13faqbe x1vvkbs xlh3980 xvmahel x1n0sxbx x1lliihq x1s928wv xhkezso x1gmr53x x1cpjm7i x1fgarty x1943h6x x4zkp8e x3x7a5m x1lkfr7t x1lbecb7 x1s688f xzsf02u")
prices_list = [prices.text.strip() for prices in prices_div]

#this is for the mileage
mileage_div = market_soup.find_all('span', class_="x1lliihq x6ikm8r x10wlt62 x1n2onr6 xlyipyv xuxw1ft")
mileage_list = [mileage.text.strip() for mileage in mileage_div]

#this is for the url
urls_div = market_soup.find_all('a', class_="x1i10hfl xjbqb8w x1ejq31n xd10rxx x1sy0etr x17r0tee x972fbf xcfux6l x1qhh985 xm0m39n x9f619 x1ypdohk xt0psk2 xe8uvvx xdj266r x11i5rnm xat24cr x1mh8g0r xexx8yu x4uap5 x18d9i69 xkhd6sd x16tdsg8 x1hl2dhg xggy1nq x1a2a7pz x1heor9g x1sur9pj xkrqix3 x1lku1pv")
urls_list = [url.get('href') for url in urls_div]

In [ ]:
mileage_list

In [ ]:
#this is to only get the main certain piece of text like city price mileage
#the first piece is for city the second peice is for abbrevation
#so its city then abbreviation
#for more info search up regular expression 
pattern = re.compile(r'(\w+(?:_\w+)?, [A-Z]{2})')

#this is for any list to store adjuste mileage entries
mileage_list2 = []

#iterate through the oringal mileage entries
#basically looking for the ones that skip or have 2 consectives entries of cities basically if it skips a mileage or has mileage twice
for item in mileage_list:
    #check each one to make sure its consecutive each city state has to have a mileage
    mileage_list2.append(item)

    #here we are checking if our current entry from the list equals to this one 
    #it has to be at least 2 items long cause if its one then no points
    #the pattern has to match the previous one
        # Check if the current item and the previous item match the pattern
            # If both items match the pattern, insert "0K miles" before the last entry
    if pattern.match(item) and len(mileage_list2) >= 2 and pattern.match(mileage_list2[-2]):
        mileage_list2.insert(-1, '0K km')
        

In [ ]:
mileage_list2

In [ ]:

#here we are going to be removing the location from the mileage list
mileage_pattern_miles = r'(\d+)K miles'
mileage_pattern_km = r'(\d+)K km'

# we need this new list so it gives us only the numbers
mileage_clean = []

#now iterating through the previous list and now adding only the clean ones into our list
for item in mileage_list2:
    #here we are going to be checking each item and find the mileage if its in km 
    match_mileage_km = re.search(mileage_pattern_miles, item)
    #same thign but miles
    match_mileage_miles = re.search(mileage_pattern_miles, item)
#so now if it finds either miles or km
    if match_mileage_miles or match_mileage_km:
        #so now if wefind either km or miles then we will append it to the miles clean list
        if match_mileage_km:
            mileage_clean.append(int(match_mileage_km.group(1)) * 1000)
        else :
            mileage_clean.append(int(match_mileage_miles.group(1)) * 1600)


In [ ]:
mileage_clean

In [ ]:

#now we are going to add everything to a list of dictionaries
vehicles_list = []

#here we are going to be interating through our lists we will create multiple dictionaries so each vehicle will be in its in own
#we need the index and item
for i, item in enumerate(titles_list):
    cars_dict = {}

    #here we need to split the title and make model becasue they are all in the saame element
    titles_split = titles_list[i].split()
    #the year is the first part of the title so we will split that [0] and then make it int
    #do same for the rest
    cars_dict["Year"] = int(titles_split[0])
    cars_dict["Make"] = titles_split[1]
    cars_dict["Model"] = titles_split[2]
    #the prolbem is we need to get rid of dollar and we just want that number
    #we need to use regular expression again
    #prices list is there because we need to access the prices of cars and then i is the specific elemnt so we use i to loop it
    cars_dict['Price'] = int(re.sub(r'[^\d.]', '', prices_list[i]))
    cars_dict["Mileage"] = mileage_clean[i]
    cars_dict["URL"] = urls_list[i]
    #once everything is done then we add it to the vehicles list
    vehicles_list.append(cars_dict)






In [ ]:
vehicles_list

In [ ]:
#now putting all the infomration on a dataframe
vehicles_df = pd.DataFrame(vehicles_list)

In [ ]:
vehicles_df

In [ ]:
#here we are going to be adding the facebook.com
vehicles_df['URL'] = 'https://www.facebook.com' + vehicles_df['URL']

In [ ]:
vehicles_df

In [ ]:
#here we are going to fix the model of the car problem 
#so first we select the model column in the dataframe
#then we convert all of it to lower so we can read all together
#model contains a specific model we wanted to name earlier
#then lower again 
#then we did the vehicles df which creates a t and f where each entry indicates whether the corresponding entry in the "Model" column matches the model variable after both have been converted to lowercase.
#vehicles_df[] basically The DataFrame is then filtered using the boolean Series. Only rows where the condition is True are included in the resulting DataFrame.
#filtered_df = vehicles_df[vehicles_df["Model"].str.lower() == model.lower()]



In [ ]:
#filtered_df

In [ ]:
#same thing for model now
#filtered_df = vehicles_df[vehicles_df["Model"].str.lower() == model.lower()]


In [ ]:
#filtered_df

In [ ]:
#here we are going to be grouping the cars by year and plotting the mean which is the avergae of the data set
#first we go into the the year section of prices part and then we want the mean
yearly_prices = vehicles_df.groupby("Year")["Price"].mean()
#this makes a bar graph

#here we are making it wider
plt.figure(figsize=(12, 6))

#this index is the x axis which is the years
#and the values is the y axis which is the average prices
plt.bar(yearly_prices.index, yearly_prices.values)


#here we are adding the average value of the car 

# Add values on top of the bars with no digits after the period
#the index is the x axis
#the value is the y axis
#zip combines them into an order paird (x,y)
#for x y in loops through each pair (years, avergae price)
for x, y in zip(yearly_prices.index, yearly_prices.values):
    #converts y into an integer removes decimal so only whole number
    y = int(y)  # Convert to an integer to remove digits after the period
    #y: The y-coordinate where the text should be placed (the average price).
    #f'{y}': The text to display, which is the average price.
    #ha='center': Horizontal alignment of the text. 'center' aligns the text horizontally in the center of the bar.
    #va='bottom': Vertical alignment of the text. 'bottom' places the text just above the top of the bar.
    plt.text(x, y, f'{y}', ha='center', va='bottom')

#here we are adding the labels
plt.xlabel('Year')
plt.ylabel('Price')
plt.title('Average Price by Year')

plt.show()


In [ ]:
yearly_prices

In [ ]:
#same thing but for the miles 
#this is to get all the missing or invalid data ones out
#filtered_df = filtered_df[filtered_df['Kilometrage'] != 0]

#here we are going to be grouping the cars by year and plotting the mean which is the avergae of the data set
#first we go into the the year section of prices part and then we want the mean
yearly_miles = vehicles_df.groupby("Year")["Mileage"].mean()
#this makes a bar graph

#here we are making it wider
plt.figure(figsize=(12, 6))

#this index is the x axis which is the years
#and the values is the y axis which is the average prices
plt.bar(yearly_miles.index, yearly_miles.values)


#here we are adding the average value of the car 

# Add values on top of the bars with no digits after the period
#the index is the x axis
#the value is the y axis
#zip combines them into an order paird (x,y)
#for x y in loops through each pair (years, avergae price)
for x, y in zip(yearly_miles.index, yearly_miles.values):
    #converts y into an integer removes decimal so only whole number
    y = int(y)  # Convert to an integer to remove digits after the period
    #y: The y-coordinate where the text should be placed (the average price).
    #f'{y}': The text to display, which is the average price.
    #ha='center': Horizontal alignment of the text. 'center' aligns the text horizontally in the center of the bar.
    #va='bottom': Vertical alignment of the text. 'bottom' places the text just above the top of the bar.
    plt.text(x, y, f'{y}', ha='center', va='bottom')

#here we are adding the labels
plt.xlabel('Year')
plt.ylabel('Mileage')
plt.title('Average Mileage by Year')
plt.show()


In [ ]:
#saving it to files
#this is the variable name to the csv file
#this saves to the current directory its in
csv_file_path = 'MarketplaceCars.csv'
# Save the DataFrame to a CSV file
#vehicles df is the infromation
#csv file path is going to be the name
#infex = false means  that the CSV file will not include the DataFrame’s index column, only the data.
vehicles_df.to_csv(csv_file_path, index=False)
